# Download and pre-process bilingual corpora

First we download and inspect both JESC and Business Scence Dialogue for the frequency of different formality registers.

## JESC:

In [1]:
# download JESC corpus

import urllib.request

link = 'https://nlp.stanford.edu/projects/jesc/data/split.tar.gz'

downloaded_filename = 'split.tar.gz'
urllib.request.urlretrieve(link, downloaded_filename)

('split.tar.gz', <http.client.HTTPMessage at 0x1b5f89c7430>)

In [2]:
# extract JESC files

import tarfile

fname = 'split.tar.gz'

if fname.endswith("tar.gz"):

    tar = tarfile.open(fname, "r:gz")
    tar.extractall()
    tar.close()

In [28]:
# JESC corpus comes already split into train/dev/test files
# each file contains Japanse sentences with their English equivalents
# these have to be split into separate Japanese and English files
# fr is for reading, fwe/fwj for writing

for fname in ['train','dev','test']:
  with open('split/' + fname, encoding='utf-8') as fr, open('split/en-' + fname + '.txt', 'w', encoding='utf-8') as fwe, open('split/ja-' + fname + '.txt', 'w', encoding='utf-8') as fwj:
    separated = zip(*(l.split('\t') for l in fr))
    unzipped = list(separated)
    # write Japanese files
    fwj.writelines(unzipped[1])
    lines = unzipped[0]
    #print(lines)
    # write English files
    for line in lines:
        fwe.write(line + '\n')
    
    

In [29]:
# convert JP train file into a list

my_file = open('split/ja-train.txt', encoding='utf-8')
all_the_lines = my_file.readlines()
items = []
for i in all_the_lines:
    items.append(i)
# print(items)
jesc_ja_train = [x[:-1] for x in items] # removes \n character
#print(jesc_ja_train)
print(len(jesc_ja_train))

# convert Eng train file into a list
my_file = open('split/en-train.txt', encoding='utf-8')
all_the_lines = my_file.readlines()
items = []
for i in all_the_lines:
    items.append(i)
# print(items)
jesc_en_train = [x[:-1] for x in items] # removes \n character
print(len(jesc_en_train))

2797388
2797388


In [30]:
# convert JP dev file into a list

my_file = open('split/ja-dev.txt', encoding='utf-8')
all_the_lines = my_file.readlines()
items = []
for i in all_the_lines:
    items.append(i)
# print(items)
jesc_ja_dev = [x[:-1] for x in items] # removes \n character
print(len(jesc_ja_dev))

# convert Eng train file into a list
my_file = open('split/en-dev.txt', encoding='utf-8')
all_the_lines = my_file.readlines()
items = []
for i in all_the_lines:
    items.append(i)
# print(items)
jesc_en_dev = [x[:-1] for x in items] # removes \n character
print(len(jesc_en_dev))

2000
2000


In [31]:
# convert JP test file into a list

my_file = open('split/ja-test.txt', encoding='utf-8')
all_the_lines = my_file.readlines()
items = []
for i in all_the_lines:
    items.append(i)
# print(items)
jesc_ja_test = [x[:-1] for x in items] # removes \n character
print(len(jesc_ja_test))

# convert Eng train file into a list
my_file = open('split/en-test.txt', encoding='utf-8')
all_the_lines = my_file.readlines()
items = []
for i in all_the_lines:
    items.append(i)
# print(items)
jesc_en_test = [x[:-1] for x in items] # removes \n character
print(len(jesc_en_test))

2000
2000


# Exploratory Data Analysis

We will analyse the distribution of different registers across both datasets. We will use a honorific heuristic as to establish the level of formality of sentences. First we compose four lists, each containing honorific keys characteristic for each register. Then, we will run string search on each sentence in both corpora files to check if it contains any of the keys which will allow us to establish the formality level of the sentece.

In [32]:
# create lists of honorific keys for each register
# informal is excluded as any sentence which does not contain the keys will be labelled as informal

polite = ['です', 'でした', 'ます', 'ました', 'ません', 'ましょう', 'でしょう'] # original honorifics used in Dugan's study
polite_with_suffixes = ['です', 'でした', 'ます', 'ました', 'ません', 'ましょう', 'でしょう', 'さん']
humble = ['へいしゃ', 'わたくし']
formal = []

## Investigate Occurence of Polite Register

## JESC:

In [6]:
# first, check the original key set:

jesc_prev_polite = []

for word in polite:
    for line in jesc_ja_train:
        if word in line:
            jesc_prev_polite.append(line)
            
print(len(jesc_prev_polite))

543009


In [33]:
jesc_polite = []

for word in polite_with_suffixes:
    for line in jesc_ja_train:
        if word in line:
            jesc_polite.append(line)
            
print(len(jesc_polite))

610963


In [30]:
# calculate how many more sentences were labelled as polite with additional honorifics keys used

jesc_difference = len(jesc_polite) - len(jesc_prev_polite)
print('Difference in sentences labelled as polite when considering honorific suffixes: ' + str(jesc_difference))

# percentage increase
jesc_increase = jesc_difference / len(jesc_ja_train) * 100
print(jesc_increase) 

Difference in sentences labelled as polite when considering honorific suffixes: 67954
2.4291946630213612


# Investigate Occurence of Humble Register

## JESC:

In [29]:
# check for humble keys in JESC

jesc_humble = []

for word in humble:
    for line in jesc_ja_train:
        if word in line:
            jesc_humble.append(line)
            
#print(jesc_humble) 

In [30]:
# calculate ratio

print(len(jesc_humble))
print(len(jesc_ja_train))

humble_ratio = len(jesc_humble)/len(jesc_ja_train) * 100
print(humble_ratio) # result too insignificant to include humble honorifics in the model as unbalanced

131
2797388
0.004682939942546404


Both corpora contain insufficient number of sentences marked as humble following our search using honorifics heuristic. This register has to be therefore excluded from the dataset.

# Investigate Occurence of Formal Register

## JESC:

In [31]:
formal = ['様', 'さま', '先生', 'せんせい', '先輩', 'せんぱい', '博士', 'はかせ', '陛下', 'へいか', '殿下', 'でんか', '閣下', 'かっか']

In [34]:
# check for formal keys in JESC train file

jesc_formal = []

for word in formal:
    for line in jesc_ja_train:
        if word in line:
            jesc_formal.append(line)

print(len(jesc_formal))            
print(jesc_formal) # some sentences contain the kanji in different context, for instance, as a part of a surname
# has to be removed from the search

39334
['そのうえで 様子を見る。 いいな?', '一般人からの様々な', '気にかけてくれてどうも ジョン だが クレアが彼女の 選択をしたのと同様に', 'この者の嫁は 織田様の ご息女でしてな。', 'これは インスタレーションの 中心から北を見た様子です', 'みんな 辞めさせて! 何様のつもりや?', '知る様になって まだひと月だろ', 'なんだ 貴様!', 'キ キンちゃん様 続きをどうぞ', '兄上様は そんな憂いを', '寝室を別にしているような 有様でして。', 'じゃなくて... 私 この度 神様になってるんですけど', '夏目様 お待ちを お願いしたいことがあるのです!', 'へっ!? 貴様もだ 隊長格。', '様子は?', '貴様らは自分たちが少年法で守られて 好き勝手に人を殺すことができる', 'また誰かがご先祖様に取られるぞ', 'それは 取り付かれた様になります', '発言する様にして下さい', 'ジュニアは母親同様 おかしくなりつつあるでしょ', '私は奥様じゃない。', 'タンクレディが王子様の居所を 突き止めた', 'お前も お黙りんす! 貴様のほうこそ黙りんす!', '氷に依存して生きている北極熊 の様な生物にとっては不都合です', 'お客様は神様なので、幸せにします', '去年発見した事を皆様に お見せしたいと思っています', 'この事に関しては様々な試みを行いました - 見えるかどうか分かりませんが -', '貴様は何者だ!', 'ソーニャは やる気ない様ね', 'か 神様 ありがとうございます', '感謝します、神様。', '子どもたちの様子が 目に浮かぶでしょう', '言わせて頂きますが ハドック様がお戻りになるのを 心待ちにしていました', 'ねえ 昔の様に あれを一度やろうか?', '奥様!', 'カイト様! これってまずいんじゃ...。', '拙者の名を呼ぶときは「様」をつけるのだ クズ蟲共', '最近ではインドネシア フロレス島でも 同様な発見がありました', 'そこに並んで建てられる様を想像できます', '貴様らが拷問される 拷問の様に', '《正ちゃんが生きててくれただけで 神様に感謝だから》', '剥いだ皮の様だ', '確認よ 日焼けし過ぎない様に', '王様の毒見役も断った', 'あんたがより理解

In [35]:
# check other keys which may be used in different context inidividually

key_to_check = 'かっか' # this was run to inspect each key from formal list individually
jesc_keys = []


for line in jesc_ja_train:
    if key_to_check in line:
        jesc_keys.append(line)

print(len(jesc_keys))            
print(jesc_keys)

12
['どがんした ヒロ兄も東京行きたかっか', 'あれから 家 帰って まだ ママ 1人で かっかしてるから➡', 'かっかっか。 けど 兄弟初の...。', '体がかっかして眠れなくなる可能性がなきにしもあらずですけれど', 'かっかっか。 これは あくまで特別だからな?', 'かっかするな', 'そうかっかしなさんな 我慢しろ我慢しろ !', 'あんたはどこが悪かっかな', 'なぜリードの野朗どもを かっかさせるんだ?', 'まあ アジは大漁じゃったけん よかっか', '体がかっかして眠れなくなる可能性がなきにしもあらずですけれど', 'でも 皮肉よね そのおかっかけと尻軽女たち いつも彼といちゃついて']


After inspecting the sentences yield with all formal honorific suffixes listed in the source, we notice that many sentences contain some keys in a different context, for example as a part of a different word:

'そのうえで 様子を見る。 いいな?' - here the kanji for honorific さま title forms the first part of a word 様子 meaning 'appearance'.

Since this formality heuristic works only if the keys used appear only in the context of honorific, any keys that are used in other context have to be removed from consideration.

Results after checking other formal honorifics individually:
'さま': used also as a part of a verb in past tense, for example: 'お疲れさまでした。'

'先生'/'せんせい': means 'teacher', here frequently used to mark occupation rather than as a honorific, for example: '私と先生の2人でも まだ足りないわけ?'

'博士'/'はかせ': honorific title used when addressing a person with very high academic expertise, here found in non-honorific context meaninng PhD students: '博士課程の学生を100人に 学位を取らせる事もできでしょう'.

The frequency of the remaining formal suffixes is too low in the dataset to consider as a separate class.

This analysis demonstrates that formal honorific titles are not suitable for the formality heuristic used in this work as the titles are frequently used in other, non-honorific context. This register will also be excluded by the classifier.

## Create a Smaller Dataset

### Training Set Preperation

In [35]:
import pandas as pd

# create dataframe from the list
train_data = pd.DataFrame({'jp_sentence': jesc_ja_train})

# add label column
train_data['label'] = 'informal' # this value will remain for informal sentences

# replace labels for polite sentences
lookup = {'polite': ['です', 'でした', 'ます', 'ました', 'ません', 'ましょう', 'でしょう', 'さん']}
for label, words in lookup.items():
    for word in words:
        train_data.loc[train_data.jp_sentence.str.contains(word, case=False, regex=False), 'label'] = label
        
print(train_data.head(20))

                        jp_sentence     label
0                           スリバン人です    polite
1   生徒がお互いの受精じゃなくて 植物の受粉に熱中してくれてるよ!  informal
2                 この雨の中 一晩中 墓地にいたい?  informal
3                    船はもう攻撃発起位置にある。  informal
4                   オリジナルの サムが居るんだぞ  informal
5                  でも ヤダから お仕事 辞めて!  informal
6            悟空は 一人旅を続けて どんどん強くなった。  informal
7                    これより 緊急隊首会を行う。  informal
8                  言い換えれば 今ここにいる部屋の  informal
9                         一体どこに居るの?  informal
10       これは名前のないカイアシ つまり甲殻プランクトンです    polite
11                   そういう勤務記録を提出しろ。  informal
12     公序良俗に反するレベルだと 思われることのほうが問題だ。  informal
13                             驚いたわ  informal
14                             動揺 ?  informal
15                 それは見事なアイデアだと思います    polite
16                              お願い  informal
17                  そこのアホより 多くを費やした  informal
18                         さあ、行くわよ!  informal
19                     くそったれ ヘンテコ野朗  informal


In [36]:
# create perfectly balanced train set with even ratio of polite and informal sentences

df_for_train_grouped = train_data.groupby("label") # group based on formality label
df_for_train_grouped.groups.values()
# sample function will randomly select rows from each class
frames_of_groups = [x.sample(df_for_train_grouped.size().min()) for y, x in df_for_train_grouped] # set the number of entries
# for each group to the size of the smallest group
balanced_train = pd.concat(frames_of_groups)
print(balanced_train.shape[0]) # check the total number of rows
num_1 = len(balanced_train[balanced_train.label == 'polite'].index) # check the number of polite sentences
print(num_1)

1135418
567709


The size of the dataset is still too large and has to be further reduced.

In [56]:
# further reduce the size of the balanced train dataset to improve training time and performance

df_for_train_grouped = balanced_train.groupby("label") # group based on formality label
df_for_train_grouped.groups.values()
# sample function will randomly select rows from each class
frames_of_groups = [x.sample(150000) for y, x in df_for_train_grouped] # set the number of entries
# for each group to the size of the smallest group
reduced_train = pd.concat(frames_of_groups)
print(reduced_train.shape[0]) # check the total number of rows
num_1 = len(reduced_train[reduced_train.label == 'polite'].index) # check the number of polite sentences
print(num_1)

300000
150000


In [57]:
from sklearn.utils import shuffle

# shuffle the dataframe
train_shuffled = shuffle(reduced_train, random_state=0)
print(train_shuffled)

                                               jp_sentence     label
2468292                                        ショータイムの始まりだ  informal
456711                                              中尉 中尉!  informal
792926                                             お前の結婚式だ  informal
1666247                                  皆さん ひと言ずつ お別れの言葉を    polite
1352327  今理解しようとしているのは、 君がアベル・ギデオンを君の病院に戻した理由だ。 心配していない...  informal
...                                                    ...       ...
679056                                           もったいない事です    polite
1938065                                           いいうちなのか?  informal
996866                              "この夜を生き抜いて 安全な生活を送ってくれ  informal
215256                              そこで彼は素晴らしいアイデアを思いつきました    polite
1022585                                         友達が何処にいるの?  informal

[300000 rows x 2 columns]


In [58]:
train_shuffled.to_csv('JESC/reduced_train.csv', index=True) # index value is changed from False to True
# as it was observed that after loading the file into ktrain data was ordered in alphabetic rather than random order

### Validation Set Preparation

In [37]:
# create dataframe from the list
dev_data = pd.DataFrame({'jp_sentence': jesc_ja_dev})

# add label column
dev_data['label'] = 'informal' # this value will remain for informal sentences

# replace labels for polite sentences
lookup = {'polite': ['です', 'でした', 'ます', 'ました', 'ません', 'ましょう', 'でしょう', 'さん']}
for label, words in lookup.items():
    for word in words:
        dev_data.loc[dev_data.jp_sentence.str.contains(word, case=False, regex=False), 'label'] = label
        
print(dev_data.head(20))

               jp_sentence     label
0          あなたは戻ったのね ハロルド?  informal
1             俺の相手は シャークだ。  informal
2           引き換えだ ある事とある物の  informal
3         もういいよ ごちそうさま ううん  informal
4      もう会社には来ないでくれ 電話もするな  informal
5                    きれいだ。  informal
6       連れて行け 殺しそうだ わかったか?  informal
7                   殺したのか!  informal
8   わぁ~! いつも すみません。 いいのよ~。    polite
9               カンパニーの元社員が  informal
10              じゃアイツラはどこ?  informal
11             相手を陥れるとか...  informal
12        必要のない子 耐えるにしている。  informal
13       犯人像とは 違和感のある見た目だが  informal
14                      あ!  informal
15         もっと詳しいの 見せますから。    polite
16               それ今持ち出すか?  informal
17     じゃあ 仕事終わったら 医局に来てよ。  informal
18  あの... 皆さんで 先 済ませてください。    polite
19       六花ちゃんは ずっと ずっとずっと  informal


In [46]:
dev_shuffled = shuffle(dev_data, random_state=0)

In [69]:
print(dev_shuffled.head(10))

                          jp_sentence     label
405                        地球にもたらされた。  informal
1190                       うるせーな 返せよッ  informal
1132                 わからないわ 私は心を読めないの  informal
731           曲は完璧なんだよ。 作り直す必要なんかねえよ。  informal
1754                        座るつもりはない!  informal
1178           ナラ・セに会わなければよい 証拠を見つけても  informal
1533                            大丈夫だよ  informal
1303                どこへ行った 加藤! 姿を見せろ!  informal
1857  どんな風に聞こえるか分かってる でも 考えられる事じゃないか?  informal
18             あの... 皆さんで 先 済ませてください。    polite


In [47]:
dev_shuffled.to_csv('JESC/validation.csv', index=True)

### Test Set Preparation

In [38]:
# create dataframe from the list
test_data = pd.DataFrame({'jp_sentence': jesc_ja_test})

# add label column
test_data['label'] = 'informal' # this value will remain for informal sentences

# replace labels for polite sentences
lookup = {'polite': ['です', 'でした', 'ます', 'ました', 'ません', 'ましょう', 'でしょう', 'さん']}
for label, words in lookup.items():
    for word in words:
        test_data.loc[test_data.jp_sentence.str.contains(word, case=False, regex=False), 'label'] = label
        
print(test_data.head(20))

                     jp_sentence     label
0                        ほぼ無関係です    polite
1                      ゲイル 酔ってる?  informal
2                           注意しろ  informal
3               最後の引き出しが 5日前にあった  informal
4                 僕が自殺し 物語を完成させる  informal
5                    国際犯罪に起きたのか?  informal
6      皆さんのiphoneで この活動に繋がってください    polite
7                 なら ついてくればいいでしょ  informal
8        それで こんなザマに 理由なんて どうでもいい  informal
9                      ああ 約束するぜ!  informal
10               ダイナマイト tnt 雷酸水銀  informal
11    売っちゃうよ! あの家。 家族みんな バラバラだよ。  informal
12                          遅いわよ  informal
13                       携帯を出したら  informal
14                     やめろ やめるんだ  informal
15         裁判所の判断によっては 子供たちは定期的に  informal
16  役者も まだ。 監督も まだ。 もちろん 企画も まだ。  informal
17                          問題ない  informal
18        駆除業者に捕まれば 二度と戻っては来られまい  informal
19                     我らの役目...。  informal


In [15]:
test_data.to_csv('JESC/test-labelled.csv', index=False)

In [18]:
# create unlabelled test set in formal expected by ktrain
no_label_test_data = pd.DataFrame({'jp_sentence': jesc_ja_test})

In [19]:
print(no_label_test_data.head(20))

                     jp_sentence
0                        ほぼ無関係です
1                      ゲイル 酔ってる?
2                           注意しろ
3               最後の引き出しが 5日前にあった
4                 僕が自殺し 物語を完成させる
5                    国際犯罪に起きたのか?
6      皆さんのiphoneで この活動に繋がってください
7                 なら ついてくればいいでしょ
8        それで こんなザマに 理由なんて どうでもいい
9                      ああ 約束するぜ!
10               ダイナマイト tnt 雷酸水銀
11    売っちゃうよ! あの家。 家族みんな バラバラだよ。
12                          遅いわよ
13                       携帯を出したら
14                     やめろ やめるんだ
15         裁判所の判断によっては 子供たちは定期的に
16  役者も まだ。 監督も まだ。 もちろん 企画も まだ。
17                          問題ない
18        駆除業者に捕まれば 二度と戻っては来られまい
19                     我らの役目...。


In [20]:
no_label_test_data.to_csv('JESC/test-unlabelled.csv', index=False)

Pre-training a ktrain BERT model with a training set the size of 300,000 sentences takes over 3 hours per epoch. As fitting multiple models for manual finetuning process will be required, the size of a set is further reduced to decrease training time to a balanced dataset of 50,000 sentences for both classes (25,000 each). This is still a suffiecient number of examples for BERT so it will not deteriorate the model's performace.

### Further Reduce the Size of Training Set for Fine-tuning Purpose

In [59]:
df_for_train_grouped = balanced_train.groupby("label") # group based on formality label
df_for_train_grouped.groups.values()
# sample function will randomly select rows from each class
frames_of_groups = [x.sample(25000) for y, x in df_for_train_grouped] # set the number of entries
# for each group to the size of the smallest group
reduced_train = pd.concat(frames_of_groups)
print(reduced_train.shape[0]) # check the total number of rows
num_1 = len(reduced_train[reduced_train.label == 'polite'].index) # check the number of polite sentences
print(num_1)

50000
25000


In [61]:
tuning_train = shuffle(reduced_train, random_state=0)
print(tuning_train)

                       jp_sentence     label
1533289         落ち着いて 私のアドバイスを忘れずに  informal
1247871               私 魔女にはなりたくない  informal
950993   翌日の4時04分に 404ドルもの賞金を渡しました    polite
1736253               大丈夫ですよ。 行こう。    polite
2185184               5年もやってれば慣れます    polite
...                            ...       ...
1042923         ヨハンセン 一定の会話は重要な...  informal
2178624   あなたに会いたいと おっしゃってる方が います。    polite
1654181              2時に 1階で会いましょう    polite
2153452               そうですか。 残念だな。    polite
62528        使っていなかったんだ 君が生まれる前からね  informal

[50000 rows x 2 columns]


In [62]:
tuning_train.to_csv('JESC/tuning_train.csv', index=True)

During the evaluation stage of the pre-trained model it was discover that the model classified all input sentences with one, 'informal' label. The inbalance in the validation dataset with most of the sentences labelled as 'informal' was identified as a possible cause which required creating a balanced dev set to improve the performance of the classifier.

# Create a balanced validation set

In [67]:
# create perfectly balanced dev set with even ratio of polite and informal sentences

df_for_dev_grouped = dev_shuffled.groupby("label") # group based on formality label
df_for_dev_grouped.groups.values()
# sample function will randomly select rows from each class
frames_of_groups = [x.sample(df_for_dev_grouped.size().min()) for y, x in df_for_dev_grouped] # set the number of entries
# for each group to the size of the smallest group
balanced_dev = pd.concat(frames_of_groups)
print(balanced_dev.shape[0]) # check the total number of rows
num = len(balanced_dev[balanced_dev.label == 'polite'].index) # check the number of polite sentences
print(num)

736
368


In [70]:
dev_shuffled = shuffle(balanced_dev, random_state=0)

In [71]:
dev_shuffled.to_csv('JESC/balanced_dev.csv', index=True)